In [1]:
# %load pso.py
import math
import numpy as np
from numba import jit
from init_particles import create_n_particles
from evaluate import evaluate, update_velocity, update_position

@jit
def error(position):
    err = 0.0
    for i in range(len(position)):
    
        xi = position[0][i]
        err += (xi * xi) - (10 * np.cos(2 * np.pi * xi)) + 10
    return err

@jit
def PSO(costFunc,bounds,maxiter, swarm_init):

    num_dimensions=len(swarm_init[0])
    err_best_g=-1                   # best error for group
    pos_best_g=[]                   # best position for group
    num_particles = len(swarm_init)
    # establish the swarm
    swarm = create_n_particles(num_particles, num_dimensions, swarm_init)
    # begin optimization loop
    i=0
    while i < maxiter:
    
        #print i,err_best_g
        # cycle through particles in swarm and evaluate fitness
        for j in range(0,num_particles):
            swarm[j]['pos_best_i'], swarm[j]['err_best_i']  = evaluate(costFunc, swarm[j])
    
            # determine if current particle is the best (globally)
            if swarm[j]['err_i'] < err_best_g or err_best_g == -1:
                pos_best_g=list(swarm[j]['position_i'])
                err_best_g=float(swarm[j]['err_i'])

        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
            swarm[j]['velocity_i'] = update_velocity(pos_best_g, swarm[j])
            swarm[j]['position_i'] = update_position(bounds, swarm[j])
        i+=1

    return pos_best_g, err_best_g

if __name__ == '__main__':
    num_particles = 15
    swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
    bounds=[(-10,10),(-10,10)]
    PSO(error,bounds,maxiter=3, swarm_init=swarm_init)
    print('working')
#     particles = create_n_particles(100, 2, bounds)
#     #evaluate(particles[0])
#     pos_best_g = particles[0]['position_i']
#     update_velocity(pos_best_g, particles[1])
#     update_position(bounds, particles[1])
    

ImportError: No module named 'init_particles'

In [ ]:
# %load particle.py
#dependencies
import random
import math
import copy # for array copying
import sys

class Particle:
    def __init__(self,x0, num_dimensions):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        self.num_dimensions = num_dimensions
        
        for i in range(0, self.num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0, self.num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0, self.num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

In [6]:
from pso import PSO, error, evaluate, update_velocity, update_position
from failed_exec import create_n_particles

bounds=[(-10,10),(-10,10)]
particles = create_n_particles(100, 2, bounds)

In [7]:
eval_time = %%timeit -o evaluate(error, particles[0])

The slowest run took 4.09 times longer than the fastest. This could mean that an intermediate result is being cached.
37.3 µs ± 23.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%load_ext line_profiler

In [21]:
%prun evaluate(error,particles[2])

In [27]:
from numba import jit
@profile
@jit
def foo():
    for i in range(100000):
        i+=1
    return
%lprun -f foo foo()

In [23]:
%lprun -f create_n_particles create_n_particles(10,2 , bounds)

In [1]:
from particle import Particle
import numpy as np
num_particles = 15
swarm_init = [np.random.uniform(-10,10, 2) for _ in range(num_particles)]
bounds=[(-10,10),(-10,10)]

particle = Particle(swarm_init[0], len(swarm_init[0]))

In [2]:
import math
def error(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi) - (10 * math.cos(2 * math.pi * xi)) + 10
  return err

In [4]:
new_time = %%timeit -o particle.evaluate(error)

2.71 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
